In [1]:
import numpy as np 
import pandas as pd
import os, zipfile

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


<h1>Preprocessing</h1>

In [2]:
# read in data
train_df = pd.read_csv('./train_onelabel.csv')
train_images_paths = ['./train_images/' + p for p in train_df.image]
train_labels = train_df['class'].values

In [3]:
# create image matrix
train_images = (load_img(p, target_size=(32, 32)) for p in train_images_paths)
train_images = [img_to_array(img)[:, :, [0]] for img in train_images]
train_images = np.array(train_images)
train_images.shape

(24204, 32, 32, 1)

In [4]:
# preprocessing of labels
classes = len(np.unique(train_labels))
train_labels = to_categorical(train_labels, classes)

In [9]:
# split data into test and train set
x_train, y_train, x_val,  y_val = train_test_split(train_images, train_labels, test_size=0.2)

In [10]:
# normalize image data

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_train /= 255
y_train /= 255

<h1>Model</h1>

In [11]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [8]:
# Initialising the CNN
classifier = Sequential()

In [9]:
# Step 1 - Convolution ---TODO: see input specification
classifier.add(Conv2D(32, (3, 3), input_shape = (32, 32,1),padding='same', activation = 'relu'))

In [10]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [11]:
# Adding a second convolutional layer+pooling layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [12]:
# Step 3 - Flattening
classifier.add(Flatten())

In [13]:
#Step 4 - Create a fully connected neural network
'''its good practise to pick a number of power of two-experimenting output_dim
128 hidden nodes in hidden layers-by experimenting'''
 

#output_dim = number of nodes in the hidden layer
classifier.add(Dense(units= 128, activation = 'relu'))

#Dropout (p was used in the old API, changed it to the new standard)
classifier.add(Dropout(rate = 0.1))

#second hidden layer
classifier.add(Dense(units = 128, activation = 'relu'))

#Dropout
#classifier.add(Dropout(p = 0.2))

#output layer
classifier.add(Dense(units = 121, activation = 'softmax')) #We have 121 categories of plankton!!

In [14]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [15]:
# fit model on training data
classifier.fit(x_train, x_val, batch_size=50, nb_epoch=15, verbose=1)

C:\Users\movetech\Anaconda2\envs\py35\lib\site-packages\keras\models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/15
19363/19363 [==============================] - 48s 2ms/step - loss: 3.6585 - acc: 0.1639
Epoch 2/15
19363/19363 [==============================] - 43s 2ms/step - loss: 2.6422 - acc: 0.3193
Epoch 3/15
19363/19363 [==============================] - 42s 2ms/step - loss: 2.3254 - acc: 0.3773
Epoch 4/15
19363/19363 [==============================] - 44s 2ms/step - loss: 2.1541 - acc: 0.4099: 0s - loss: 2.1566 - acc:
Epoch 5/15
19363/19363 [==============================] - 54s 3ms/step - loss: 2.0127 - acc: 0.4416
Epoch 6/15
19363/19363 [==============================] - 45s 2ms/step - loss: 1.9156 - acc: 0.4591
Epoch 7/15
19363/19363 [==============================] - 42s 2ms/step - loss: 1.8262 - acc: 0.4875: 1s - loss: 1
Epoch 8/15
19363/19363 [==============================] - 41s 2ms/step - loss: 1.7659 - acc: 0.4928
Epoch 9/15
19363/19363 [==============================] - 42s 2ms/step - loss: 1.7069 - acc: 0.5112
Epoch 10/15
19363/19363 [==============================] - 4

In [16]:
score = classifier.evaluate(y_train, y_val, verbose=0)

In [17]:
score

[1.6067611749840531, 0.55009295603776365]

<h1>Save Model</h1>

In [18]:
from keras.models import model_from_json

# serialize model to JSON
best_model_classification_json = classifier.to_json()
with open("best_model_3.json", "w") as json_file:
    json_file.write(best_model_classification_json)
# serialize weights to HDF5
classifier.save_weights("model_best.h5")
print("Saved model to disk")

Saved model to disk


<h1>Load Model</h1>

In [12]:
from keras.models import model_from_json

# load json and create model
json_file = open('best_model_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
classifier = model_from_json(loaded_model_json)
# load weights into new model
classifier.load_weights("model_best.h5")
print("Loaded model from disk")

Loaded model from disk


<h1>Prediction - First Approach</h1>

In [13]:
# read in data
test_df = pd.read_csv('./sample.csv')
test_images_paths = ['./test_images/' + p for p in test_df['image']]
test_images_names = [p for p in test_df['image']]

In [18]:
# create image matrix
test_images = (load_img(p, target_size=(32, 32)) for p in test_images_paths)
test_images = [img_to_array(img)[:, :, [0]] for img in test_images]
test_images = np.array(test_images)
test_images.shape

(6132, 32, 32, 1)

In [22]:
# normalize image data
test_images = test_images.astype('float32')
test_images /= 255

In [48]:
result = classifier.predict(test_images)

In [50]:
labels = []

for count, i in enumerate(result):
    labels.append(np.argmax(result[count]))

In [54]:
from collections import OrderedDict

#Create Final csv for the competition
csv_list =  pd.DataFrame( OrderedDict( { 'image': pd.Series(test_images_names), 'class': pd.Series(labels) } ) )
csv_list.to_csv('sample1.csv',index=False)

In [64]:
results1 = pd.read_csv('./sample1.csv')

<h1>Prediction - Second Approach </h1>

In [59]:
import os
import numpy as np
from keras.preprocessing import image
from collections import OrderedDict

def find_category(path):
    
    test_image = image.load_img(path, target_size = (32, 32),grayscale = True)
    test_image = image.img_to_array(test_image)
  
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    
    #training_set.class_indices
    
    
    category = int(np.where(result[0]!=0)[0][0])
    return category
    

images = []
categories = []
path = 'test_images/'

for count,i in enumerate(os.listdir(path)):
    
    images.append(str(i)) #save image name
    
    image_path = path+str(i)

    category = find_category(image_path)
    
    categories.append(category) #saves the category of the image

In [60]:
images = []
categories = []
path = 'test_images/'

for count,i in enumerate(os.listdir(path)):
    
    images.append(str(i)) #save image name
    
    image_path = path+str(i)

    category = find_category(image_path)
    
    categories.append(category) #saves the category of the image

In [61]:
#Create Final csv for the competition
csv_list =  pd.DataFrame( OrderedDict( { 'image': pd.Series(images), 'class': pd.Series(categories) } ) )
csv_list.to_csv('sample2.csv',index=False)

In [65]:
results2 = pd.read_csv('./sample2.csv')

In [72]:
comparison = results1.merge(results2, on='image')

In [81]:
print('Number of different results: ' + str(len(comparison[comparison['class_x'] != comparison['class_y']])))ka

Number of different results: 5738
